# Весенний проект python. Курс "Науки о данных.

## Программа для поиска букмекерских "вилок"

Данная программа собирает информацию на основании введенных данных с двух букмекерских контор (конторы выбраны заранее), анализирует их и выдает букмекерские "вилки" (если таковые имеются). 
Букмекерская вилка - это ситуация, в которой можно сделать две ставки на одно и то же событие так, что вы точно получите прибыль. То есть, своего рода арбитраж.

### Пример работы: 
Допустим, есть некоторый матч между Командой 1 и Командой 2 с коэффициентами на победу k1 и k2 соответственно. На втором сайте эти же коэффициенты равны m1 и m2. Программа анализирует, можно ли на первом сайте поставить на первый исход (победа первой команды), а на другом сайте на противоположный так, чтобы при любом исходе остаться в плюсе. 

### В программе были использованы следующие библиотеки (для облегчения оценивания):

1) Selenium и BeautifulSoup, то есть, Веб-скреппинг 

2) Анализ JSON файла для поиска коэффициентов 

3) Создание и работа с таблицами pandas для представления данных и последующего анализа.

4) Определенно 75+ строк

5) Использовалась библиотека numpy

5) "Вау" эффект всегда остается на ваше усмотрение) 

6) В конце даже есть небольшой интерактивчик

### Что необходимо сделать для работы программы: 

1) Импортировать все необходимые библиотеки 

2) Установить Selenium в ваш Jupyter 

3) Запустить все функции в указанном порядке 

4) Если будут ошибки в анализе данных со странице (первая ячейка), скорее всего, ошибка в введении названии спорта/страны/лиги. Пожалуйста, проверьте соответствуют ли введенные данные тому, что представлено на сайте. 

5) Установить pip install Pillow (это правда очень важно)

6) Если написано "Пожалуйста, скопируйте/введите точно/еще что-то такое", то, пожалуйста, вводите точно, постарайтесь не ошибаться в символах. 


### Замечания:

1) Может быть проблема сортировки по странам (к примеру, теннис в Лиге Ставок сортируется не по странам), поэтому с ним возникнут проблемы.

2) Снова с Лигой Ставок: могут возникнуть проблемы с Селениумом (в частности, со словом "Кубок", так как вероятно может быть выбран не тот турнир, который вы ожидаете). Вполне возможно, что к моменту проверки эта проблема исчезнет, но будьте готовы. Если вы правда хотите посмотреть, как все работает, лучше выбирать относительно уникальные названия (к примеру, вряд ли много названий "Лига 2"). 

3) Извините, что в конце придется много копировать и вставлять - не придумал способа уравнять названия некоторых команд. 

4) Про проверку присутствия игр (или лиг) на обоих сайтах я вспомнил в последний момент, поэтому, пожалуйста, не берите слишком редкие. 

In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
import requests
import json
import time         
from selenium import webdriver    
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
from PIL import Image



In [30]:
print("Пожалуйста, посмотрите названия лиг в начале")
url = "https://www.ligastavok.ru"
sport = input("Введите вид спорта: ")   
country = input("Введите страну: ")  
browser = webdriver.Chrome("D:\chromedriver.exe")
browser.get(url)
um = browser.find_element_by_xpath("//span[contains(./text(),'" +sport.strip()+ "')]")
actions = ActionChains(browser) 
um.click()
ul = browser.find_element_by_xpath("//span[contains(./text(),'" +country.strip()+ "')]")
ul.click()

Пожалуйста, посмотрите названия лиг в начале
Введите вид спорта: Футбол
Введите страну: Англия


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Windows NT 10.0.17134 x86_64)


In [31]:
def sel(url,sport, country, liga):
    browser = webdriver.Chrome("D:\chromedriver.exe")
    browser.get(url)
    um = browser.find_element_by_xpath("//span[contains(./text(),'" +sport.strip()+ "')]")
    actions = ActionChains(browser) 
    um.click()
    
    ul = browser.find_element_by_xpath("//span[contains(./text(),'" +country.strip()+ "')]")
    
    ul.click()
    
    uk = browser.find_element_by_xpath("//span[contains(./text(),'" +liga.strip()+ "')]")
    
    uk.click()
    
    j = browser.current_url
    return j
   


liga = input("Введите название турнира (как оно дано на сайте): ")   
h = sel(url,sport,country,liga)


Введите название турнира (как оно дано на сайте): Лига 2


In [32]:
def js(j):
    r = requests.get(j)
    t = r.text
    obj = BeautifulSoup(t, features="html")
    a = obj.html.head.findAll('script')[1].text[14:-1]
 
    z = a.replace("'", "\"") 
    dic = json.loads(z)
    return dic
k = js(h)

In [33]:
if 'popular' in k['BUILineProvider']['deps']:
    idd = k['BUILineProvider']['deps']['popular']
elif 'my-line' in k['BUILineProvider']['deps']:
    idd = k['BUILineProvider']['deps']['my-line']

In [34]:
team1=[]
team2=[]
win1 = []
x = []
win2=[]
iddd = []
res = []
res2=[]
for i in idd:
    
    team1 = k['BUILineProvider']['events'][str(i)]['event']['team1']
    team2 = k['BUILineProvider']['events'][str(i)]['event']['team2']
    
    if 'main' in k['BUILineProvider']['events'][str(i)]['outcomesWinner']:
        win1 = k['BUILineProvider']['events'][str(i)]['outcomesWinner']['main']['outcomes']['_1']['value']
        x = k['BUILineProvider']['events'][str(i)]['outcomesWinner']['main']['outcomes']['x']['value']
        win2 = k['BUILineProvider']['events'][str(i)]['outcomesWinner']['main']['outcomes']['_2']['value']
        res.append([i,team1,team2,win1,x,win2])

    
    for j in range(1,4):
    
        if 'main' in k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]:


            less1 = k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]['main']['outcomes']['less']['title']

            lessv = k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]['main']['outcomes']['less']['value']
            gross1 = k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]['main']['outcomes']['gross']['title']
            grossv = k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]['main']['outcomes']['gross']['value']
            value = k['BUILineProvider']['events'][str(i)]['outcomesTotal'+str(j)]['main']['outcomes']['less']['adValue']
            res2.append([i,team1,team2,lessv,less1,float(value),gross1,grossv])


WIN = pd.DataFrame(res, columns = ['id','Команда 1','Команда 2','Победа 1','Ничья','Победа 2']).sort_values(by='Команда 1').reset_index().drop(columns=['index'])
Total_1 = pd.DataFrame(res2, columns = ['id','Команда 1','Команда 2','КМ','Меньше','Тотал','Больше','КБ']).sort_values(by='Команда 1').reset_index().drop(columns=['index'])


In [35]:
WIN

,id,Команда 1,Команда 2,Победа 1,Ничья,Победа 2
0,11317465,Карлайл Юнайтед,Линкольн Сити,3.10,3.25,2.20
1,11317470,Кембридж Юнайтед,Суиндон Таун,2.70,3.15,2.55
2,11317464,Колчестер Юнайтед,Гримсби Таун,1.72,3.50,4.70
3,11317467,Кру Александра,Йовил Таун,1.86,3.40,4.00
4,11317468,Мансфилд Таун,Моркам,1.58,3.85,5.40
5,11317472,Нортгемптон Таун,Маклсфилд Таун,1.99,3.40,3.55
6,11317473,Ноттс Каунти,Милтон Кинс Донс,3.20,3.40,2.10
7,11317466,Ньюпорт Каунти,Бери,3.40,3.25,2.09
8,11317474,Порт Вейл,Стивенидж,2.23,3.15,3.15
9,11317469,Эксетер Сити,Кроли Таун,1.86,3.40,4.00


In [36]:
Total_1

,id,Команда 1,Команда 2,КМ,Меньше,Тотал,Больше,КБ
0,11317465,Карлайл Юнайтед,Линкольн Сити,1.79,мен,2.5,бол,1.95
1,11317465,Карлайл Юнайтед,Линкольн Сити,2.55,мен,2.0,бол,1.48
2,11317465,Карлайл Юнайтед,Линкольн Сити,1.43,мен,3.0,бол,2.70
3,11317470,Кембридж Юнайтед,Суиндон Таун,1.74,мен,2.5,бол,2.02
4,11317470,Кембридж Юнайтед,Суиндон Таун,2.43,мен,2.0,бол,1.52
5,11317470,Кембридж Юнайтед,Суиндон Таун,3.20,мен,1.5,бол,1.32
6,11317464,Колчестер Юнайтед,Гримсби Таун,3.15,мен,1.5,бол,1.33
7,11317464,Колчестер Юнайтед,Гримсби Таун,1.75,мен,2.5,бол,2.01
8,11317464,Колчестер Юнайтед,Гримсби Таун,2.43,мен,2.0,бол,1.52
9,11317467,Кру Александра,Йовил Таун,1.43,мен,3.0,бол,2.70


Теперь получим информацию о турнире со второго сайта: 


Выберите подходящий Вам способ (через pandas или Selenium, можете попробовать оба :-) ) 

In [37]:
url2 = "https://www.leon.ru/"
r = requests.get(url2)
t = r.text
obj = BeautifulSoup(t, features="html")    
a = []
b = obj.findAll('a', class_ = "sportItem")
for i in b:
    a.append(i.span.string)
sports = a[1:]
sites = ['Soccer', 'Tennis', 'IceHockey','Basketball','Volleyball','Handball','MMA','Boxing','AustralianRules','Baseball','Darts','Cricket','TableTennis','RugbyLeague','Snooker','Floorball','Footzal']
spsites = dict(list(zip(sports,sites)))

In [38]:
url2it = "https://www.leon.ru/event/"+spsites[sport.strip()]
r2 = requests.get(url2it)
t2 = r.text
obj = BeautifulSoup(t2, features="html")    
pd.options.display.max_colwidth = 100
ress = []
b = obj.findAll('li', class_ = "")
for i in b: 
    if i.a['href']!=None:
        ccc = i.a['href']
    if i.span!=None:
        lll = i.span.string

    ress.append([ccc,lll])
     
Site = pd.DataFrame(ress, columns = ['Ссылка на страницу','2'])

In [39]:
k=[]
kk=[]
res3=[]
for i in b:
    if i.span != None: 
        k.append(i.span.string)
for i in k: 
    if i != None: 
        if i.startswith(country.strip()):
            res3.append(i)
Tournir = pd.DataFrame(res3,columns=['Турнир'])


In [40]:
v=pd.merge(left=Tournir, right=Site, left_on='Турнир', right_on='2', how='left')
vv = v.drop(columns = ['2'])
vvv= vv.drop_duplicates()
sports_and_sites = vvv.reset_index().drop(columns=['index'])
sports_and_sites


,Турнир,Ссылка на страницу
0,Англия - Премьер-лига,/events/Soccer/281474976710692-England-Premier-League
1,Англия - Вторая Лига,/events/Soccer/281474976710728-England-League-2
2,Англия - Национальная Лига,/events/Soccer/281474976710730-England-National-League
3,"Англия - Национальная Лига, Север",/events/Soccer/281474976710894-England-National-League-North
4,Англия - Первая Лига,/events/Soccer/281474976710725-England-League-1
5,Англия - Чемпионшип,/events/Soccer/281474976710719-England-Championship
6,"Англия - Южная Лига, Премьер Дивизион, Юг",/events/Soccer/281474976721851-England-Southern-League-Premier-Division-South
7,"Англия - Королевский Однодневный Кубок, Лондон",/events/Cricket/281474976716288-England-Royal-London-One-Day-Cup
8,Англия - Суперлига,/events/RugbyLeague/281474976711083-England-Super-League


In [41]:
s2 = sports_and_sites.set_index("Турнир", drop=True)
Name = input("Введите название турнира (из таблицы выше - то, которое вы выбрали до этого): ") 
Site_Name = s2.loc[Name.strip()]['Ссылка на страницу']
url_liga = 'https://www.leon.ru/events/'+Site_Name

Введите название турнира (из таблицы выше - то, которое вы выбрали до этого): Англия - Вторая Лига	


Через Selenium:

In [125]:
url2it = "https://www.leon.ru/events/"+spsites[sport]
browser = webdriver.Chrome("D:\chromedriver.exe")
browser.get(url2it)
print("В открытом браузере выберите нужную страну и турнир (в меню слева - чтобы URL поменялся)")
print("Когда выберете нужный Вам турнир, запустите следующую ячейку")

В открытом браузере выберите нужную страну и турнир (в меню слева - чтобы URL поменялся)
Когда выберете нужный Вам турнир, запустите следующую ячейку


In [126]:
url_liga = browser.current_url
url_liga

'https://www.leon.ru/events/Soccer/281474976710728-England-League-2'

#### Тем или иным образом мы получили необходимую ссылку

In [42]:
r = requests.get(url_liga)
t = r.text
obj = BeautifulSoup(t, features="html") 
n = obj.findAll('script', type="application/ld+json")
games = []
for i in n: 
    a = i.string
    zz = a.replace("'", "\"") 
    dicc = json.loads(zz)
    if 'name' in dicc: 
        games.append(dicc['name'])
Games = pd.DataFrame(games,columns=['Игры'])

In [43]:
m = obj.findAll('script', type="text/javascript")
k2=m[-10].string.split('initialEvents:')[1].split(',\n')[0]
z2 = k2.replace("'", "\"") 
dic2 = json.loads(z2)
n=[]
for i in range(len(dic2['events'])):
    o = dic2['events'][i]['markets'][0]['runners']
    l = dic2['events'][i]['name'].split(' - ')
    f = dic2['events'][i]['url']
    for j in range(len(o)):
        b = o[j]['price']
        c = o[j]['name']
        n.append([f,l[0],l[1],b,c])


In [44]:
q = pd.DataFrame(n,columns = ['url','team1','team2','K','Is'])
table = pd.pivot_table(q, columns = 'Is', values = 'K', index = ['team1','team2','url'])
second_buk  = pd.DataFrame(table.to_records())

second_buk



,team1,team2,url,1,2,X
0,Карлайл Юнайтед,Линкольн Сити,/events/Soccer/281474976710728-England-League-2/281474982121133-Carlisle-United-Lincoln-City,3.21,2.27,3.47
1,Кембридж Юнайтед,Суиндон Таун,/events/Soccer/281474976710728-England-League-2/281474982121132-Cambridge-United-Swindon-Town,2.78,2.65,3.30
2,Колчестер Юнайтед,Гримсби Таун,/events/Soccer/281474976710728-England-League-2/281474982121135-Colchester-United-Grimsby-Town,1.74,5.05,3.75
3,Кру Александра,Йовил Таун,/events/Soccer/281474976710728-England-League-2/281474982121136-Crewe-Alexandra-Yeovil-Town,1.88,4.35,3.60
4,Мансфилд Таун,Моркам,/events/Soccer/281474976710728-England-League-2/281474982121138-Mansfield-Town-Morecambe,1.60,5.95,4.05
5,Нортгемптон Таун,Маклесфилд Таун,/events/Soccer/281474976710728-England-League-2/281474982121140-Northampton-Town-Macclesfield-Town,2.02,3.70,3.65
6,Ноттс Каунти,Милтон Кинс Донс,/events/Soccer/281474976710728-England-League-2/281474982121141-Notts-County-Milton-Keynes-Dons,3.35,2.18,3.55
7,Ньюпорт Каунтри,Бери,/events/Soccer/281474976710728-England-League-2/281474982121139-Newport-County-Bury,3.49,2.15,3.48
8,Порт Вэйл,Стивенидж,/events/Soccer/281474976710728-England-League-2/281474982121142-Port-Vale-Stevenage,2.29,3.28,3.35
9,Транмир Роверс,Форест Грин Роверс,/events/Soccer/281474976710728-England-League-2/281474982121143-Tranmere-Rovers-Forest-Green-Rovers,2.41,3.10,3.31


In [45]:
Win = WIN.set_index("Команда 1",drop = True )

SB = second_buk.set_index("team1",drop = True )


In [46]:
Win

,id,Команда 2,Победа 1,Ничья,Победа 2
Команда 1,,,,,
Карлайл Юнайтед,11317465,Линкольн Сити,3.10,3.25,2.20
Кембридж Юнайтед,11317470,Суиндон Таун,2.70,3.15,2.55
Колчестер Юнайтед,11317464,Гримсби Таун,1.72,3.50,4.70
Кру Александра,11317467,Йовил Таун,1.86,3.40,4.00
Мансфилд Таун,11317468,Моркам,1.58,3.85,5.40
Нортгемптон Таун,11317472,Маклсфилд Таун,1.99,3.40,3.55
Ноттс Каунти,11317473,Милтон Кинс Донс,3.20,3.40,2.10
Ньюпорт Каунти,11317466,Бери,3.40,3.25,2.09
Порт Вейл,11317474,Стивенидж,2.23,3.15,3.15


In [47]:
SB

,team2,url,1,2,X
team1,,,,,
Карлайл Юнайтед,Линкольн Сити,/events/Soccer/281474976710728-England-League-2/281474982121133-Carlisle-United-Lincoln-City,3.21,2.27,3.47
Кембридж Юнайтед,Суиндон Таун,/events/Soccer/281474976710728-England-League-2/281474982121132-Cambridge-United-Swindon-Town,2.78,2.65,3.30
Колчестер Юнайтед,Гримсби Таун,/events/Soccer/281474976710728-England-League-2/281474982121135-Colchester-United-Grimsby-Town,1.74,5.05,3.75
Кру Александра,Йовил Таун,/events/Soccer/281474976710728-England-League-2/281474982121136-Crewe-Alexandra-Yeovil-Town,1.88,4.35,3.60
Мансфилд Таун,Моркам,/events/Soccer/281474976710728-England-League-2/281474982121138-Mansfield-Town-Morecambe,1.60,5.95,4.05
Нортгемптон Таун,Маклесфилд Таун,/events/Soccer/281474976710728-England-League-2/281474982121140-Northampton-Town-Macclesfield-Town,2.02,3.70,3.65
Ноттс Каунти,Милтон Кинс Донс,/events/Soccer/281474976710728-England-League-2/281474982121141-Notts-County-Milton-Keynes-Dons,3.35,2.18,3.55
Ньюпорт Каунтри,Бери,/events/Soccer/281474976710728-England-League-2/281474982121139-Newport-County-Bury,3.49,2.15,3.48
Порт Вэйл,Стивенидж,/events/Soccer/281474976710728-England-League-2/281474982121142-Port-Vale-Stevenage,2.29,3.28,3.35


## Проверка вилки

Наконец-то мы пришли к тому, что и должна делать наша программа - к проверке наличия вилок. Что мы делаем для этого? Немного математики: 

Рассмотрим выражение: 1/K1+1/K2+..+1/Kn = B, где В - значение вилки. 

В случае с проверкой на победителя матча, коэффициентов может быть 3, с тоталом - 2. 

Мы проверяем, меньше ли В единицы, то есть, нам нужны наибольшие значения коэффициентов (из двух контор). 

Почему именно это? 

Если мы поставим 1$, то заработает k1 или k2 , то есть, цена одного выигранного доллара для нас есть 1/k1 поставленный. Если сумма поставленных долларов меньше единицы, то мы точно выйдем в +, так как для нас реальная стоимость выигранного доллара меньше 1. 

Тогда сумма равна SUM = (1/((K1)**(B)) ** N, где N - сумма

In [48]:
while(True):
    coeff = []
    print("Будьте внимательны - выберите один и тот же матч")
    Comanda1 = input("Пожалуйста, введите Команду 1 из первой таблицы (Win) (просто скопируйте): ")
    Comanda2 = input("Пожалуйста, введите Команду 1 из второй таблицы (SB) (аналогично): ")
    k1 = [Win.loc[Comanda1.strip()]['Победа 1'],SB.loc[Comanda2.strip()]['1']]
    k11 = list(map(float, k1))
    K1 = max(k11)
    k2 = [Win.loc[Comanda1.strip()]['Победа 2'],SB.loc[Comanda2.strip()]['2']]
    k22 = list(map(float, k2))
    K2 = max(k22)
    coeff.append(K1)
    coeff.append(K2)
    try:
        x = [Win.loc[Comanda1.strip()]['Ничья'],SB.loc[Comanda2.strip()]['X']]
        xx = list(map(float, x))
        X = max(xx)

        coeff.append(X)
    except :
        print("Нет ничьи")
    coef = np.array(coeff)
    coef = coef**(-1)
    summ = np.sum(coef)
    if summ<=1: 
        print("Ура! Мы нашли вилку! Теперь давайте рассчитаем суммы, которые нужно поставить")
        money = float(input("Введите имеющуюся сумму:"))
        stavki = coef*money/summ
        print(coef)
    else:
        print("К сожалению, вилки здесь нет") 
        print("Вы хотите поискать еще?")
    answer = input("Да/Нет: " )
    if answer == "Нет":
        break
    else:
        clear_output()
    coef

Будьте внимательны - выберите один и тот же матч
Пожалуйста, введите Команду 1 из первой таблицы (Win) (просто скопируйте): Карлайл Юнайтед	
Пожалуйста, введите Команду 1 из второй таблицы (SB) (аналогично): Карлайл Юнайтед	
К сожалению, вилки здесь нет
Вы хотите поискать еще?
Да/Нет: Нет


Это были таблицы по Победе/Проигрышу.

Рассмотрим те же варианты для Тотал! 

In [22]:
Comanda_total = input("Пожалуйста, введите Команду 1 из второй таблицы: ")

Total = SB.loc[Comanda_total.strip()]['url']
url_total = "https://www.leon.ru/events/"+Total
rr = requests.get(url_total)
ttot = rr.text
objtot = BeautifulSoup(ttot, features="html") 
mtot = objtot.findAll('script', type="text/javascript")
ktot=mtot[-10].string.split('initialEvents:')[1].split(',\n')[0]



Пожалуйста, введите Команду 1 из второй таблицы: Бостон Селтикс	


In [23]:
ntot = []
ztot = ktot.replace("'", "\"") 
dictot = json.loads(ztot)
for i in range(len(dictot['events'][0]['markets'])):
    otot = dictot['events'][0]['markets'][i]['runners']
    ltot = dictot['events'][0]['name']
    ytot = dictot['events'][0]['markets'][i]['name']
    if ytot == 'Тотал':
        for j in range(len(otot)):
            btot = otot[j]['price']
            ctot = otot[j]['name'].split("(")
            if ctot[0].strip()=='Меньше':
                ttt='мен'
            if ctot[0].strip()=='Больше': 
                ttt='бол'
            ntot.append([ltot,btot,ttt,float(ctot[1].strip(")"))])

qtot = pd.DataFrame(ntot,columns = ['Название матча','K','Б/М','Тотал'])
tabletot = pd.pivot_table(qtot, columns = 'Б/М', values = 'K', index = ['Название матча','Тотал'])
Total_2  = pd.DataFrame(tabletot.to_records())

Total_2

,Название матча,Тотал,бол,мен
0,Бостон Селтикс - Индиана Пэйсерс,203.5,2.03,1.79


In [24]:
Total_1.reset_index().drop(columns=['index'])


,id,Команда 1,Команда 2,КМ,Меньше,Тотал,Больше,КБ


In [25]:
Comanda_total1 = input("Пожалуйста, введите Команду 1 из первой таблицы: ")
Total_11 = Total_1.set_index("Команда 1",drop = True)
Total_Itog = Total_11.loc[Comanda_total1.strip()]

Пожалуйста, введите Команду 1 из первой таблицы: 123


KeyError: 'the label [123] is not in the [index]'

In [142]:
MergeTableTotal=pd.merge(left=Total_Itog, right=Total_2, left_on='Тотал', right_on='Тотал', how='right')
MTT = MergeTableTotal.drop(columns=['id','Команда 2'])
MergeTableTotall = MTT.dropna()
MergeTableTotall

,КМ,Меньше,Тотал,Больше,КБ,Название матча,бол,мен
0,1.79,мен,2.5,бол,1.95,Карлайл Юнайтед - Линкольн Сити,2.05,1.86
1,2.55,мен,2.0,бол,1.48,Карлайл Юнайтед - Линкольн Сити,1.55,2.63
2,1.43,мен,3.0,бол,2.70,Карлайл Юнайтед - Линкольн Сити,2.95,1.46


К сожалению, я уже не успел доделать эту часть, но логика подобна тому, что мы делали с победами и поражениями. Надеюсь, всё работало хорошо!

### А теперь добавим немножко интерактива (совсем немного)

In [26]:
def m1(b):
    im = Image.open(requests.get("http://enottone.ru/images/media/all_enot/enot-poloskun_enot_tone_7.jpg", stream=True).raw)
    img = im.resize((460, 360), Image.ANTIALIAS)

    return display(img)
def m2(b):
    im2 = Image.open(requests.get("https://cs8.pikabu.ru/post_img/big/2016/12/28/1/1482881197192578721.jpg", stream=True).raw)
    img2 = im2.resize((460, 360), Image.ANTIALIAS)

    return display(img2)
def surp(b):
    im3 = Image.open(requests.get("https://w-dog.ru/wallpapers/6/18/372783628575012.jpg", stream=True).raw)
    img3 = im3.resize((720, 540), Image.ANTIALIAS)
    return display(img3)

In [28]:
CAT_b = widgets.Button(description = "Кот")
ENOT_b = widgets.Button(description = "Енот")
SURP_b =   widgets.Button(description = "Долгожданный")
display(CAT_b,ENOT_b,SURP_b)
    
CAT_b.on_click(m2)
ENOT_b.on_click(m1)
SURP_b.on_click(surp)

Button(description='Кот', style=ButtonStyle())

Button(description='Енот', style=ButtonStyle())

Button(description='Долгожданный', style=ButtonStyle())

#### Надеюсь, этот проект вам хотя бы чуть-чуть понравился) 

##### Мне он чуть-чуть точно понравился
